<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-family:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
SQL: Subqueries
              
</p>
</div>

Data Science Cohort Live NYC May 2022
<p>Phase 1: Topic 7</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>
    
   

Remember: 

SELECT queries return a new table.

Sometimes:

- Query a table.
- Filter/select/... on that query
- Return result of querying on the first query.

First query is a **subquery**.

Let's get into it with our handy dandy flights database!

In [1]:
import sqlite3
import numpy as np
import pandas as pd

# connect to flights database
con = sqlite3.connect('data/flights.db')

Subqueries can be used to great effect in a variety of circumstances:
    
- Selecting from a table created by a subquery 
    - Subquery in FROM statement
- Filtering on a subquery.
    - Subquery in a WHERE statement
- Filtering in a SELECT statement.


Example of a query containing a subquery:

Get the average of the highest altitude airports in United States, Russia, and China:

- Select airport name, city, and altitude for the highest airport in each country
- Then filter this on the country.
- Then get the average of the altitudes of the resultant airports.

Step by step.

In [2]:
%%capture subquery
pd.read_sql("""

SELECT name, 
     city,
     country,
     MAX(CAST(altitude as float)) AS max_altitude
     
     FROM airports
     
     GROUP BY country
     

""", con)

In [3]:
subquery()

,name,city,country,max_altitude
0,FOB Sharana,Sharan,Afghanistan,7400.0
1,Tirana Rinas,Tirana,Albania,126.0
2,Tamanrasset,Tamanrasset,Algeria,4518.0
3,Fitiuta Airport,Fiti\\'uta,American Samoa,110.0
4,Lubango,Lubango,Angola,5778.0
...,...,...,...,...
235,Jerusalem,Jerusalem,West Bank,2485.0
236,Smara Airport,Smara,Western Sahara,350.0
237,Sanaa Intl,Sanaa,Yemen,7216.0
238,Kasompe,Kasompe,Zambia,4636.0


This is a table produced by the query. We now want to filter this table on country.

- Can select FROM results of above table and filter WHERE conditon is true.
- Subquery creates intermediate table for us.

In [4]:
%%capture query

pd.read_sql("""

SELECT *
    
FROM 

(SELECT name, 
     city,
     country,
     MAX(CAST(altitude as float)) AS max_altitude
     
     FROM airports
     
     GROUP BY country
     
     ) AS high_alt_airp
     
 WHERE high_alt_airp.country IN ("United States", "Russia", "China")



""", con)

In [5]:
query()

,name,city,country,max_altitude
0,Yading Daocheng,Daocheng,China,14472.0
1,Irkutsk-2,Irkutsk,Russia,13411.0
2,Silverton,Silverton,United States,9308.0


Now I select the mean altitude across the three countries.

In [6]:
%%capture query

pd.read_sql("""

SELECT AVG(high_alt_airp.max_altitude) AS max_altitude_avg
    
FROM 

(SELECT name, 
     city,
     country,
     MAX(CAST(altitude as float)) AS max_altitude
     
     FROM airports
     
     GROUP BY country
     
     ) AS high_alt_airp
     
 WHERE high_alt_airp.country IN ("United States", "Russia", "China")



""", con)

In [7]:
# Average altitude of the highest airports in Russia, United States, and China
query()

,max_altitude_avg
0,12397.0


One can create, in principle, a new table of derived columns and do selections and filtering on that new table.

Let's take a look at our routes table again.

In [8]:
pd.read_sql("""

SELECT * 
     FROM routes
     LIMIT 5


""", con)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2


For each airport I want:
- the count of routes that leaves from there.
- the count of routes that arrive there.

Many ways to do this. But I'll make full use of subqueries and joins here.

In [9]:
departure_table = pd.read_sql("""

SELECT source, COUNT(source) as departure_count
     FROM routes
     
     GROUP BY source
     
""", con)

In [10]:
departure_table

,source,departure_count
0,AAE,9
1,AAL,20
2,AAN,2
3,AAQ,3
4,AAR,8
...,...,...
3404,ZUH,60
3405,ZUM,2
3406,ZVK,3
3407,ZYI,15


In [11]:
arrival_table = pd.read_sql("""

SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     
     GROUP BY dest
     
""", con)

In [12]:
arrival_table

,dest,arrival_count
0,AAE,9
1,AAL,21
2,AAN,2
3,AAQ,3
4,AAR,8
...,...,...
3413,ZUH,59
3414,ZUM,2
3415,ZVK,3
3416,ZYI,15


We can link these the arrival tables and destination tables together:
- Keep records where source and destination are in both tables.
- What operation?

INNER JOIN ON SUBQUERY RESULTS!

In [13]:
%%capture full_result

pd.read_sql("""

SELECT dep.source AS airport_code,
        dep.departure_count,
        arriv.arrival_count 
        
FROM

(SELECT source, COUNT(source) as departure_count
     FROM routes
     
     GROUP BY source) AS dep

INNER JOIN 

(SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     
     GROUP BY dest) AS arriv

ON dep.source = arriv.dest
     
""", con)

In [14]:
full_result()

,airport_code,departure_count,arrival_count
0,AAE,9,9
1,AAL,20,21
2,AAN,2,2
3,AAQ,3,3
4,AAR,8,8
...,...,...,...
3397,ZUH,60,59
3398,ZUM,2,2
3399,ZVK,3,3
3400,ZYI,15,15


This result is a derived table telling us useful information for each airport and keeping relevant records.

- We can filter or aggregate on this derived table as we see fit.


- Get all airports with high amount of routes servicing it: 
    - departure and arrival count both greater than 100.
- Order by arrival and then deperature count both in descending order.

Help this hapless data science instructor out!

In [15]:
%%capture filtered_result

pd.read_sql("""

SELECT dep.source AS airport_code,
        dep.departure_count,
        arriv.arrival_count 
        
FROM

(SELECT source, COUNT(source) as departure_count
     FROM routes
     
     GROUP BY source) AS dep

INNER JOIN 

(SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     
     GROUP BY dest) AS arriv

ON dep.source = arriv.dest
     
""", con)

In [16]:
filtered_result()

,airport_code,departure_count,arrival_count
0,AAE,9,9
1,AAL,20,21
2,AAN,2,2
3,AAQ,3,3
4,AAR,8,8
...,...,...,...
3397,ZUH,60,59
3398,ZUM,2,2
3399,ZVK,3,3
3400,ZYI,15,15


- Complex as subqueries pile on. 
- There are ways to organize subqueries: common table expressions
- Will make code readable.

But first: other ways subqueries can be used.

#### Subqueries in the WHERE statement

- Can use the result of a subquery to filter another selection.

I want all route records where the departure airport has a departure route count greater than 200.

In [ ]:
%%capture where_subquery1
pd.read_sql('''


SELECT *
            
FROM routes

WHERE source IN (SELECT source 
     FROM routes
     GROUP BY source
     HAVING COUNT(source) > 200) 
''', con)

In [ ]:
where_subquery1()

Subquerying made this query possible.

#### The Semijoin

- Using a subquery from one table to filter another table.

Example:

Get the airport name, code, country, and altitude for airports with more than 200 routes originating from that airport.

In [ ]:
%%capture semijoin
pd.read_sql('''

SELECT *
FROM airports

WHERE code IN (
SELECT source 
     FROM routes
     GROUP BY source
     HAVING COUNT(source) > 300 
)
''', con)

In [ ]:
semijoin()

#### The antijoin

Just the negation of the semijoin:

In [ ]:
%%capture antijoin

pd.read_sql('''
SELECT *
FROM airports

WHERE NOT code IN (
SELECT source 
     FROM routes
     GROUP BY source
     HAVING COUNT(source) > 300 
)
''', con)

In [ ]:
antijoin()

#### Subqueries in the SELECT statement

Get altitude of airport side by side with average airport altitude in the respective country.

In [ ]:
%%capture select_subquery

pd.read_sql('''
SELECT country,
name, 
CAST(altitude as int) AS altitude ,

(SELECT AVG(CAST(altitude as int))
FROM airports
GROUP BY 
country) AS avg_alt

FROM airports


''', con)

In [ ]:
select_subquery()

#### Common Table Expressions (CTEs)

Useful for when:

- Multiple subqueries floating around.
- Reusing same subqueries over and over again in same statement.

Cleans code up, reduces error, makes readable.

CTE syntax:

WITH subquery1_name AS (subquery1 statement),
<br>
WITH subquery2_name AS (subquery2 statement), 
<br>
...., 
<br>
WITH subquery_n_name AS (subquery_n statement)
<br>
<br>
MAIN QUERY

In [ ]:
%%capture filtered_result

pd.read_sql("""

WITH s1 AS (
     SELECT source, COUNT(source) as departure_count
     FROM routes
     GROUP BY source),
     
     s2 AS ( 
     SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     GROUP BY dest
     )


SELECT source, departure_count, arrival_count
FROM
s1 INNER JOIN s2
ON s1.source = s2.dest

     
""", con)

In [ ]:
filtered_result()

CTEs make the primary query easy to read.

- Subqueries can just be referenced by looking it up in the CTE.